In [0]:
# 🔹 Definizione dei widget
dbutils.widgets.text("SPOTIFY_CLIENT_ID", "")
dbutils.widgets.text("SPOTIFY_CLIENT_SECRET", "")
dbutils.widgets.text("SPOTIFY_REFRESH_TOKEN", "")

In [0]:
# REFRESH SPOTIFY TOKEN (1 hour expiration time)

import base64
import requests

# I use parameters instead of secrets as env variable because i am in Databricks Free Edition that has sevral limtiaitons

SPOTIFY_CLIENT_ID = dbutils.widgets.get("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = dbutils.widgets.get("SPOTIFY_CLIENT_SECRET")
SPOTIFY_REFRESH_TOKEN = dbutils.widgets.get("SPOTIFY_REFRESH_TOKEN")
def get_spotify_token():
    """
    Get access token from Spotify using refresh token.
    
    Returns:
        str: Access token if successful
        dict: Error response if failed
    """
    auth_str = f"{SPOTIFY_CLIENT_ID}:{SPOTIFY_CLIENT_SECRET}"
    b64_auth = base64.b64encode(auth_str.encode()).decode()

    try:
        resp = requests.post(
            "https://accounts.spotify.com/api/token",
            headers={"Authorization": f"Basic {b64_auth}"},
            data={
                "grant_type": "refresh_token",
                "refresh_token": SPOTIFY_REFRESH_TOKEN
            },
            timeout=10
        )

        if resp.status_code == 200:
            auth_data = resp.json()
            print("Token Generated")
            return auth_data['access_token']
        else:
            print({'statusCode': resp.status_code, 'body': resp.text})
            return {'statusCode': resp.status_code, 'body': resp.text}
            
    except requests.RequestException as e:
        return {'statusCode': 500, 'body': f'Request failed: {str(e)}'}
    

fresh_token = get_spotify_token()

In [0]:
dbutils.jobs.taskValues.set("SPOTIFY_TOKEN", fresh_token)